<a href="https://colab.research.google.com/github/Thakuei/PD3/blob/main/20250505_optimization_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 91.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import pulp

In [3]:
df = pd.read_csv('/content/2023_summary.csv')
df

,TIME,太陽光EZAグリッド電力(W),太陽光EZAバッテリ電力(W),バッテリEZAグリッド側電力(W),バッテリEZAバッテリ側電力(W),パワコンCT電流（00.00A）,直流母線計測電圧（000.0V),制御電源電流(0.00A),バッテリSOC(%),系統2EZAグリッド電力,系統2EZAバッテリ電力,Lib2EZAグリッド電力,Lib2EZAバッテリ電力,コテージ102のAC負荷,コテージ102の制御装置負荷,コテージ102の負荷,太陽光パネル(1.7kw)の発電量,余剰電力,month_day
0,2023-07-01,4.852606,-86.755739,72.911891,-10.093095,137.032917,3203.048426,48.726250,53.647338,0.000000,0.000000,291.504471,261.684700,438.923068,156.072538,594.995606,-81.903133,-676.898739,07-01
1,2023-07-02,-233.330705,-328.695651,90.128468,11.452993,136.552037,3295.052546,46.061852,42.654653,0.000000,0.000000,-128.745128,-196.637132,449.946137,151.776222,601.722360,-562.026356,-1163.748716,07-02
2,2023-07-03,-259.001047,-353.513534,5.504693,-79.693773,136.543843,3308.222060,46.922569,48.055949,0.000000,0.000000,153.588359,77.085283,451.717352,155.230279,606.947632,-612.514581,-1219.462212,07-03
3,2023-07-04,-239.866073,-335.675925,43.283929,-40.398407,136.120208,3333.041644,47.204792,44.216620,0.000000,0.000000,9.828723,-56.881064,453.694323,157.335536,611.029859,-575.541998,-1186.571857,07-04
4,2023-07-05,-43.538637,-138.243471,92.458683,5.871695,138.565394,3286.237685,47.474144,45.746088,0.000000,0.000000,66.996143,6.756168,455.358818,156.011320,611.370138,-181.782108,-793.152246,07-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,2023-11-26,-110.427856,-186.598799,110.100132,28.480332,144.610440,3313.491968,43.283125,37.749653,-420.700116,-512.515694,79.617677,4.599056,479.165531,143.418287,622.583818,-297.026655,-919.610473,11-26
149,2023-11-27,-46.246310,-122.669248,291.282032,209.163129,147.966227,3278.645347,43.366551,38.348125,-295.389375,-387.375116,65.946339,5.897522,485.128781,142.183540,627.312322,-168.915558,-796.227879,11-27
150,2023-11-28,56.890474,-22.668429,-93.061150,-177.024276,148.799583,3266.380394,43.066574,42.343264,1.124931,-68.426875,40.050704,10.818905,486.036042,140.671813,626.707855,34.222045,-592.485810,11-28
151,2023-11-29,58.693667,-18.995356,119.333253,31.452038,148.902176,3255.575509,43.375347,46.699329,9.897593,-56.383333,33.256615,7.581527,484.762277,141.211718,625.973995,39.698311,-586.275684,11-29


In [4]:
df['TIME'] = pd.to_datetime(df['TIME'])
df.set_index('TIME', inplace=True)

In [5]:
df = df[["コテージ102の負荷", "太陽光パネル(1.7kw)の発電量", "余剰電力"]]
df

,コテージ102の負荷,太陽光パネル(1.7kw)の発電量,余剰電力
TIME,,,
2023-07-01,594.995606,-81.903133,-676.898739
2023-07-02,601.722360,-562.026356,-1163.748716
2023-07-03,606.947632,-612.514581,-1219.462212
2023-07-04,611.029859,-575.541998,-1186.571857
2023-07-05,611.370138,-181.782108,-793.152246
...,...,...,...
2023-11-26,622.583818,-297.026655,-919.610473
2023-11-27,627.312322,-168.915558,-796.227879
2023-11-28,626.707855,34.222045,-592.485810


# 最適化１

In [6]:
def optimize_one(df, charge_max=300, discharge_max=300):

    T = range(len(df))  # 時刻のリスト
    surplus_power = dict(enumerate(df['余剰電力']))

    # --- モデル作成 ---
    model = pulp.LpProblem("Battery_Operation", pulp.LpMinimize)

    # --- 変数定義 ---
    charge = pulp.LpVariable.dicts("charge", T, lowBound=0, upBound=charge_max, cat='Continuous')
    discharge = pulp.LpVariable.dicts("discharge", T, lowBound=0, upBound=discharge_max, cat='Continuous')
    z = pulp.LpVariable.dicts("z", T, lowBound=0, cat='Continuous')

    # --- 制約 ---
    for t in T:
        model += z[t] >= surplus_power[t] + charge[t] - discharge[t]
        model += z[t] >= -(surplus_power[t] + charge[t] - discharge[t])

    # --- 目的関数 ---
    model += pulp.lpSum([z[t] for t in T])

    # --- ソルバー実行 ---
    status = model.solve()
    print(f"Status: {pulp.LpStatus[status]}")

    # --- 結果まとめ ---
    result = pd.DataFrame(index=df.index)
    result['余剰電力'] = df['余剰電力']
    result['充電量'] = [charge[t].varValue for t in T]
    result['放電量'] = [discharge[t].varValue for t in T]
    result['系統売買量'] = [
        surplus_power[t] + result['充電量'][t] - result['放電量'][t] for t in T
    ]

    return result

In [7]:
result = optimize_one(df)
display(result)

Status: Optimal


<ipython-input-6-8d1e7e102b57>:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  surplus_power[t] + result['充電量'][t] - result['放電量'][t] for t in T


,余剰電力,充電量,放電量,系統売買量
TIME,,,,
2023-07-01,-676.898739,300.0,0.0,-376.898739
2023-07-02,-1163.748716,300.0,0.0,-863.748716
2023-07-03,-1219.462212,300.0,0.0,-919.462212
2023-07-04,-1186.571857,300.0,0.0,-886.571857
2023-07-05,-793.152246,300.0,0.0,-493.152246
...,...,...,...,...
2023-11-26,-919.610473,300.0,0.0,-619.610473
2023-11-27,-796.227879,300.0,0.0,-496.227879
2023-11-28,-592.485810,300.0,0.0,-292.485810


# 最適化２

In [8]:
def optimize_two(
    df, charge_max=300, discharge_max=300, battery_capacity=1000, initial_soc=10, self_discharge_rate=10
):
    T = range(len(df))
    surplus_power = dict(enumerate(df['余剰電力']))

    model = pulp.LpProblem("Battery_Operation_with_Self_Discharge", pulp.LpMinimize)

    # 変数定義
    charge = pulp.LpVariable.dicts("charge", T, lowBound=0, upBound=charge_max, cat='Continuous')
    discharge = pulp.LpVariable.dicts("discharge", T, lowBound=0, upBound=discharge_max, cat='Continuous')
    soc = pulp.LpVariable.dicts("soc", T, lowBound=0, upBound=battery_capacity, cat='Continuous')
    z = pulp.LpVariable.dicts("z", T, lowBound=0, cat='Continuous')

    # 初期残量制約
    model += soc[0] == initial_soc

    for t in T:
        print(f"[t={t}] surplus={surplus_power[t]}, charge_max={charge_max}")
        # 系統売買量の絶対値を線形化
        net_power = surplus_power[t] + charge[t] - discharge[t]
        model += z[t] >= net_power
        model += z[t] >= -net_power

        # SoC更新式（自己放電あり）
        if t < len(df) - 1:
            model += soc[t+1] == soc[t] + charge[t] - discharge[t] - self_discharge_rate

    # 目的関数：系統売買量の絶対値合計を最小化
    model += pulp.lpSum([z[t] for t in T])

    # ソルバー実行
    status = model.solve()
    print("Solve status:", pulp.LpStatus[status])

    # 結果出力
    result = pd.DataFrame(index=df.index)
    result['余剰電力'] = df['余剰電力']
    result['充電量'] = [charge[t].varValue for t in T]
    result['放電量'] = [discharge[t].varValue for t in T]
    result['残量(SOC)'] = [soc[t].varValue for t in T]
    result['系統売買量'] = [surplus_power[t] + result['充電量'][t] - result['放電量'][t] for t in T]

    return result

In [9]:
result = optimize_two(df)
display(result)

[t=0] surplus=-676.8987392060109, charge_max=300
[t=1] surplus=-1163.7487156108743, charge_max=300
[t=2] surplus=-1219.4622123706638, charge_max=300
[t=3] surplus=-1186.5718572998844, charge_max=300
[t=4] surplus=-793.1522455837569, charge_max=300
[t=5] surplus=-1289.2440826175423, charge_max=300
[t=6] surplus=-1313.5492047811213, charge_max=300
[t=7] surplus=-784.622507033305, charge_max=300
[t=8] surplus=-970.732499292899, charge_max=300
[t=9] surplus=-1173.4488402083705, charge_max=300
[t=10] surplus=-1060.883841726737, charge_max=300
[t=11] surplus=-905.2398079364448, charge_max=300
[t=12] surplus=-1012.7309450842876, charge_max=300
[t=13] surplus=-1002.0974907769072, charge_max=300
[t=14] surplus=-1343.3925015228738, charge_max=300
[t=15] surplus=-1364.8073232944705, charge_max=300
[t=16] surplus=-1523.402770098235, charge_max=300
[t=17] surplus=-1398.9236090806412, charge_max=300
[t=18] surplus=-898.7159233672654, charge_max=300
[t=19] surplus=-1221.9227560725187, charge_max=300


<ipython-input-8-5825988db86b>:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  result['系統売買量'] = [surplus_power[t] + result['充電量'][t] - result['放電量'][t] for t in T]


,余剰電力,充電量,放電量,残量(SOC),系統売買量
TIME,,,,,
2023-07-01,-676.898739,10.0,0.0,10.0,-666.898739
2023-07-02,-1163.748716,0.0,0.0,10.0,-1163.748716
2023-07-03,-1219.462212,10.0,0.0,0.0,-1209.462212
2023-07-04,-1186.571857,10.0,0.0,0.0,-1176.571857
2023-07-05,-793.152246,20.0,0.0,0.0,-773.152246
...,...,...,...,...,...
2023-11-26,-919.610473,0.0,0.0,290.0,-919.610473
2023-11-27,-796.227879,150.0,0.0,280.0,-646.227879
2023-11-28,-592.485810,300.0,0.0,420.0,-292.485810


# 最適化３

In [ ]:
def optimize_two(
    df, charge_max=300, discharge_max=300, battery_capacity=1000, initial_soc=10, self_discharge_rate=10
):
    T = range(len(df))
    surplus_power = dict(enumerate(df['余剰電力']))

    model = pulp.LpProblem("Battery_Operation_with_Self_Discharge", pulp.LpMinimize)

    # 変数定義
    charge = pulp.LpVariable.dicts("charge", T, lowBound=0, upBound=charge_max, cat='Continuous')
    discharge = pulp.LpVariable.dicts("discharge", T, lowBound=0, upBound=discharge_max, cat='Continuous')
    soc = pulp.LpVariable.dicts("soc", T, lowBound=0, upBound=battery_capacity, cat='Continuous')
    z = pulp.LpVariable.dicts("z", T, lowBound=0, cat='Continuous')

    # 初期残量制約
    model += soc[0] == initial_soc

    for t in T:
        print(f"[t={t}] surplus={surplus_power[t]}, charge_max={charge_max}")
        # 系統売買量の絶対値を線形化
        net_power = surplus_power[t] + charge[t] - discharge[t]
        model += z[t] >= net_power
        model += z[t] >= -net_power

        # SoC更新式（自己放電あり）
        if t < len(df) - 1:
            model += soc[t+1] == soc[t] + charge[t] - discharge[t] - self_discharge_rate

    # 目的関数：系統売買量の絶対値合計を最小化
    model += pulp.lpSum([z[t] for t in T])

    # ソルバー実行
    status = model.solve()
    print("Solve status:", pulp.LpStatus[status])

    # 結果出力
    result = pd.DataFrame(index=df.index)
    result['余剰電力'] = df['余剰電力']
    result['充電量'] = [charge[t].varValue for t in T]
    result['放電量'] = [discharge[t].varValue for t in T]
    result['残量(SOC)'] = [soc[t].varValue for t in T]
    result['系統売買量'] = [surplus_power[t] + result['充電量'][t] - result['放電量'][t] for t in T]

    return result